<font color=purple><h1>**Safety Gear Monitoring System for Construction Workers**</h1></font>

<font color=Zircon><h4>**IMPORT DEPENDENCIES**</h4></font>

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score,confusion_matrix


In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

<font color=Zircon><h4>**DATA GENERATOR**</h4></font>

In [7]:
# Set up the data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_labels = pd.read_csv('normal_train/_annotations.csv')
# Convert the column to strings
train_labels['class'] = train_labels['class'].astype(str)
train_generator = train_datagen.flow_from_dataframe(
    train_labels,
    directory='normal_train',
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 4091 validated image filenames belonging to 5 classes.


In [8]:
import tensorflow as tf

def smooth_l1(y_true, y_pred):
    diff = tf.abs(y_true - y_pred)
    less_than_one = tf.cast(tf.less(diff, 1.0), "float32")
    l1_loss = (less_than_one * 0.5 * diff ** 2) + (1 - less_than_one) * (diff - 0.5)
    return tf.reduce_mean(l1_loss)

<font color=Zircon><h4>**TRAINING THE MODEL**</h4></font>

In [9]:
# Load the pre-trained ResNet-50 model and freeze the base layers
base_model = tf.keras.applications.ResNet50(
    include_top=False, weights='imagenet', input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

# Add new fully connected layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Compile the model with binary cross-entropy loss and Adam optimizer
model = Model(inputs=base_model.input, outputs=predictions)

#freeze Layers
for layer in base_model.layers:
    layer.trainable = False

EPOCHS = 1
INIT_LR = 1e-1
BS = 128
from tensorflow.keras.optimizers.experimental import SGD
opt = SGD(learning_rate=INIT_LR, weight_decay=INIT_LR / EPOCHS)


model.compile(optimizer=opt, loss=smooth_l1,
              metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20)

2023-04-24 21:33:44.442267: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/20
127/127 [==============================] - 151s 1s/step - loss: 0.0842 - accuracy: 0.8000
Epoch 2/20
127/127 [==============================] - 143s 1s/step - loss: 0.0802 - accuracy: 0.8000
Epoch 3/20
127/127 [==============================] - 191s 1s/step - loss: 0.0802 - accuracy: 0.8000
Epoch 4/20
127/127 [==============================] - 236s 2s/step - loss: 0.0802 - accuracy: 0.8000
Epoch 5/20
127/127 [==============================] - 235s 2s/step - loss: 0.0802 - accuracy: 0.8000
Epoch 6/20
127/127 [==============================] - 241s 2s/step - loss: 0.0802 - accuracy: 0.8000
Epoch 7/20
127/127 [==============================] - 236s 2s/step - loss: 0.0802 - accuracy: 0.8000
Epoch 8/20
127/127 [==============================] - 230s 2s/step - loss: 0.0802 - accuracy: 0.8000
Epoch 9/20
127/127 [==============================] - 235s 2s/step - loss: 0.0802 - accuracy: 0.8000
Epoch 10/20
127/127 [==============================] - 228s 2s/step - loss: 0.0802 - accura

+++++++<font color=Zircon><h4>**TESTING THE MODEL**</h4></font>

In [10]:
# Load the test CSV file with filenames
t_test_df = pd.read_csv('normal_test/_annotations.csv')

In [11]:
# Create a generator for the test images
t_test_datagen = ImageDataGenerator(rescale=1./255)
t_test_generator = t_test_datagen.flow_from_dataframe(
    t_test_df,
    directory="normal_test",
    x_col='filename',
    y_col=None,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False)

# Use the trained model to make predictions on the test data
preds = model.predict_generator(t_test_generator, steps=None)
# Remove any missing filenames from the test CSV file
missing_filenames = set(t_test_generator.filenames) - set(t_test_df['filename'])
if missing_filenames:
    print(f'Removing {len(missing_filenames)} missing filenames from the test CSV file.')
    t_test_df = t_test_df[~t_test_df['filename'].isin(missing_filenames)]

Found 958 validated image filenames.


/home/u190421/tmp/ipykernel_1472226/1578501580.py:14: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  preds = model.predict_generator(t_test_generator, steps=None)
2023-04-24 23:06:39.475318: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [12]:
li = []
for i in preds.ravel():
    if i>0.5:
        li.append(1)
    else:
        li.append(0)
     
# Create a DataFrame with the filenames and predictions
df = pd.DataFrame({'filename': t_test_generator.filenames[:len(preds)], 'class': li})

test_inst_df=pd.merge(df, t_test_df, on='filename', how='inner')

<font color=Zircon><h4>**SAVING THE MODEL**</h4></font>

In [14]:
model.save('final_gear_resnet/resnet.h5')